In [ ]:
## feature engineering -- create function to pull out coefficients/features
## add a gradient descent
def lasso_optimization():
    eval_metrics_dictionary = {'mean_r2':[],'mean_MAPE':[],'mean_MSE':[],'std_r2':[],'std_MAPE':[],'std_MSE':[],'score':[], 'alpha':[], 'coeff_used':[], 'coefficients':[], 'intercept':[]}

    # initialize and train model with (default value) alpha = 0.5
    for i in [0.1, 1, 10,100,1000, 10000, 33000,66000, 100000, 330000,660000,1000000, 3300000,6600000,10000000,33000000,66000000,100000000]:

        model = Lasso(alpha=i, max_iter=int(2000))
        model.fit(X_train,y_train)
        
        ## EVALUATION METRICS
        # you should really use cross_val_score rather than testing against the test set
        ## use cross validation for 
        score = cross_validate(model, X_train, y_train, scoring=('neg_mean_absolute_percentage_error', 'r2', 'neg_mean_squared_error'), cv=3)
        
        ## pull out the scores
        mean_MAPE = -1*np.mean(score['test_neg_mean_absolute_percentage_error'])
        mean_r2 = np.mean(score['test_r2'])
        mean_MSE = -1*np.mean(score['test_neg_mean_squared_error'])
        std_MAPE = np.std(score['test_neg_mean_absolute_percentage_error'])
        std_r2 = np.std(score['test_r2'])
        std_MSE = np.std(score['test_neg_mean_squared_error'])
        coeff_used = np.sum(model.coef_!=0)
        coefficients = model.coef_
        intercept = model.intercept_

        dicKeys = ['mean_MAPE','mean_r2','mean_MSE','std_MAPE','std_r2','std_MSE','score','alpha','coeff_used', 'coefficients', 'intercept'] 
        dicValues = [mean_MAPE, mean_r2, mean_MSE, std_MAPE, std_r2, std_MSE, len(score), i, coeff_used, coefficients, intercept]
        for (keyi, vali) in zip(dicKeys, dicValues):
            eval_metrics_dictionary[keyi].append(vali)

        #eval_metrics_dictionary['mean_MAPE'].append(mean_MAPE)
        #eval_metrics_dictionary['mean_r2'].append(mean_r2)
        #eval_metrics_dictionary['mean_MSE'].append(mean_MSE)
        #eval_metrics_dictionary['std_MAPE'].append(std_MAPE)
        #eval_metrics_dictionary['std_r2'].append(std_r2)
        #eval_metrics_dictionary['std_MSE'].append(std_MSE)
        #eval_metrics_dictionary['score'].append(len(score))
        #eval_metrics_dictionary['alpha'].append(i)
        #eval_metrics_dictionary['coeff_used'].append(coeff_used)

    df_framework = pd.DataFrame(eval_metrics_dictionary)
    # * by the r2_score that is closest to 1 --- you want to minimise the values of the rest, dont need to include RMSE, you overweight that value
    df_framework['combined_score'] = abs(abs(1 - df_framework['mean_r2'])*df_framework['mean_MSE']*df_framework['mean_MAPE'])
    df_bestvalue = df_framework[df_framework['combined_score']==min(df_framework['combined_score'])]
    return df_bestvalue
#return df_framework.sort_values('combined_score')

In [ ]:
hi = lasso_optimization()

In [ ]:
# take list of coefficients - if more than zero 
# keep the name of the poly feature
# keep the non-zero poly features values
featurenames = list(p.get_feature_names_out(X.columns))
coefficient_list = list(hi['coefficients']) 

### creature a dictionary of features and coefficients and then turn into dataframe
feature_coefficient_dict = {'feature':[], 'coefficient':[]}
for (feaI, coeI) in zip(featurenames,coefficient_list): 
    if coeI != 0: 
        feature_coefficient_dict['feature'].append(feaI)
        feature_coefficient_dict['coefficient'].append(coeI)
        
pd.DataFrame(feature_coefficient_dict)
feature_selection = feature_coefficient_dict['feature']
#### with the coefficients create a new np.array to 
ridge_Xtrain = np.array(pd.DataFrame(X_train, columns= p.get_feature_names_out(X.columns))[feature_selection])

np.array(pd.DataFrame(X_train, columns= p.get_feature_names_out(X.columns))[feature_selection])

In [ ]:
def ridge_optimization(number_of_trials):
    # input to study should be dataset and number of iterations
    eval_metrics_dictionary = {'mean_r2':[],'mean_MAPE':[],'mean_MSE':[],'std_r2':[],'std_MAPE':[],'std_MSE':[],'score':[]}
    ridge_opt_dictionary = { 'alpha': [], 'intercept' : [], 'tol': [], 'solver':[], 'no_of_coefficients':[]}

    def objective(trial):
        ## HYPER PARAMETER TUNING
        alpha = trial.suggest_float("alpha", 0, 1000)
        intercept = trial.suggest_categorical("fit_intercept", [True, False])
        tol = trial.suggest_float("tol", 0.001, 10, log=True)
        solver = trial.suggest_categorical("solver", ["auto", "svd","cholesky", "lsqr", "saga", "sag"])

        ## MODEL SELECTION
        ## Create Model
        model = Ridge(alpha=alpha,fit_intercept=intercept,tol=tol,solver=solver)
        ## Fit Model
        model.fit(ridge_Xtrain, y_train)
        
        # Function parameters for cross_val_score; 
        #   - estimator - The model object to use to fit the data
        #   - X - The data to fit the model on
        #   - y - The target of the model
        #   - scoring - The error metric to use
        #   - cv - The number of splits to use
        
        # error score to use; 
        #   - accuracy
        #   - balanced_accuracy
        #   - roc_auc
        #   - f1
        #   - neg_mean_absolute_error
        #   - neg_root_mean_squared_error
        #   - r2
        
        ## EVALUATION METRICS
        # you should really use cross_val_score rather than testing against the test set
        score = cross_validate(model, X_train, y_train, scoring=('neg_mean_absolute_percentage_error', 'r2', 'neg_mean_squared_error'), cv=3)
        mean_MAPE = -1*np.mean(score['test_neg_mean_absolute_percentage_error'])
        mean_r2 = np.mean(score['test_r2'])
        mean_MSE = -1*np.mean(score['test_neg_mean_squared_error'])
        std_MAPE = np.std(score['test_neg_mean_absolute_percentage_error'])
        std_r2 = np.std(score['test_r2'])
        std_MSE = np.std(score['test_neg_mean_squared_error'])
        
        """
        y_pred_train= ridge_regressor.predict(X_test)
            
        Evaluation metrics;
        - R squared
        - MSE
        - RMSE
        - MAPE
        
        infun_MSE = mean_squared_error(y_test,y_pred_train)
        infun_RMSE = math.sqrt(infun_MSE)
        infun_r2 = r2_score(y_test, y_pred_train) 
        infun_MAPE = mean_absolute_percentage_error(y_test, y_pred_train) 
        max_coeff = ridge_regressor.coef_.round(3)
        eval_metrics_dictionary['r2'].append(infun_r2)
        eval_metrics_dictionary['MSE'].append(infun_MSE)
        eval_metrics_dictionary['RMSE'].append(infun_RMSE)
        eval_metrics_dictionary['MAPE'].append(infun_MAPE)
        """
        coeff_used = np.sum(model.coef_!=0)

        eval_metrics_dictionary['mean_MAPE'].append(mean_MAPE)
        eval_metrics_dictionary['mean_r2'].append(mean_r2)
        eval_metrics_dictionary['mean_MSE'].append(mean_MSE)
        eval_metrics_dictionary['std_MAPE'].append(std_MAPE)
        eval_metrics_dictionary['std_r2'].append(std_r2)
        eval_metrics_dictionary['std_MSE'].append(std_MSE)
        eval_metrics_dictionary['score'].append(len(score))
        
        ridge_opt_dictionary['alpha'].append(alpha)
        ridge_opt_dictionary['intercept'].append(intercept)
        ridge_opt_dictionary['tol'].append(tol)
        ridge_opt_dictionary['solver'].append(solver)
        ridge_opt_dictionary['no_of_coefficients'].append(coeff_used)

        return mean_MAPE

    study2 = optuna.create_study(study_name="RidgeRegression Optimization")
    study2.optimize(objective, n_trials=number_of_trials)
    df_framework = pd.DataFrame(eval_metrics_dictionary)
    # * by the r2_score that is closest to 1 --- you want to minimise the values of the rest, dont need to include RMSE, you overweight that value
    df_framework['combined_score'] = abs(abs(1 - df_framework['mean_r2'])*df_framework['mean_MSE']*df_framework['mean_MAPE'])
    df_bestvalue = df_framework[df_framework['combined_score']==min(df_framework['combined_score'])]
    return df_bestvalue